In [1]:
import pandas as pd
# Arreglo de los dataframes para el sistema de recomendacion 
df_reviews = pd.read_csv("reviews_eda.csv")
df_games = pd.read_csv("games_eda.csv")
df_games=df_games[["item_id","title"]]
df_reviews = df_reviews[["user_id","item_id","recommend"]]
df = pd.merge(df_reviews,df_games, on="item_id", how="inner")
df = df[["user_id","title","recommend"]]
df_filtered = df[df['recommend'] == True]
df = df.astype(str)
df.to_csv("df_MLOps.csv", index= False)

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Construye una matriz de usuarios y juegos
# Encuentra el índice del usuario en la matriz
# Calcula la similitud de coseno entre los usuarios
# Obtén las similitudes de coseno para el usuario dado
# Encuentra los juegos que el usuario no ha jugado
# Calcula las puntuaciones de recomendación sumando las similitudes de usuarios para los juegos no jugados
# Ordena las recomendaciones por puntuación descendente
# Limita las recomendaciones a los primeros 5 juegos

def recomendacion_usuario(user_id :str):
    
    user_game_matrix = pd.crosstab(df['user_id'], df['title'])

    try:
        user_index = user_game_matrix.index.get_loc(user_id)
    except KeyError:
        print(f"El usuario {user_id} no está presente en los datos.")
        return None

    cosine_similarities = cosine_similarity(user_game_matrix, user_game_matrix)
    similar_users = cosine_similarities[user_index]
    games_played = user_game_matrix.loc[user_id]
    unrated_games = games_played[games_played == 0].index
    recommendation_scores = user_game_matrix.loc[:, unrated_games].multiply(similar_users, axis=0).sum(axis=0)
    recommendations = recommendation_scores.sort_values(ascending=False).index.tolist()
    top_recommendations = recommendations[:5]

    return top_recommendations


usuario_a_recomendar = "76561197970982479"
recomendacion_usuario('76561197970982479')

['Team Fortress 2',
 "Garry's Mod",
 'Counter-Strike: Global Offensive',
 'Left 4 Dead 2',
 'Terraria']